In [16]:
import pandas as pd
import numpy as np

# Example US states and counties/cities for demonstration
states = [
    ("US-CA", ["Los Angeles", "San Diego", "San Francisco"]),
    ("US-TX", ["Harris", "Dallas", "Tarrant"]),
    ("US-NY", ["Kings", "Queens", "New York"]),
    ("US-FL", ["Miami-Dade", "Broward", "Palm Beach"]),
    ("US-IL", ["Cook", "DuPage", "Lake"]),
    ("US-PA", ["Philadelphia", "Allegheny", "Montgomery"]),
    ("US-OH", ["Cuyahoga", "Franklin", "Hamilton"]),
    ("US-GA", ["Fulton", "Gwinnett", "Cobb"]),
    ("US-NC", ["Mecklenburg", "Wake", "Guilford"]),
    ("US-MI", ["Wayne", "Oakland", "Macomb"])
]

cities = {
    "Los Angeles": ["Los Angeles", "Long Beach", "Glendale"],
    "San Diego": ["San Diego", "Chula Vista", "Oceanside"],
    "San Francisco": ["San Francisco", "Daly City", "South San Francisco"],
    "Harris": ["Houston", "Pasadena"],
    "Dallas": ["Dallas", "Irving"],
    "Tarrant": ["Fort Worth", "Arlington"],
    "Kings": ["Brooklyn", "Williamsburg"],
    "Queens": ["Queens", "Astoria"],
    "New York": ["Manhattan", "Harlem"],
    "Miami-Dade": ["Miami", "Hialeah"],
    "Broward": ["Fort Lauderdale", "Pembroke Pines"],
    "Palm Beach": ["West Palm Beach", "Boca Raton"],
    "Cook": ["Chicago", "Cicero"],
    "DuPage": ["Naperville", "Wheaton"],
    "Lake": ["Waukegan", "Mundelein"],
    "Philadelphia": ["Philadelphia", "Chestnut Hill"],
    "Allegheny": ["Pittsburgh", "Monroeville"],
    "Montgomery": ["Norristown", "Pottstown"],
    "Cuyahoga": ["Cleveland", "Parma"],
    "Franklin": ["Columbus", "Dublin"],
    "Hamilton": ["Cincinnati", "Norwood"],
    "Fulton": ["Atlanta", "Sandy Springs"],
    "Gwinnett": ["Lawrenceville", "Duluth"],
    "Cobb": ["Marietta", "Smyrna"],
    "Mecklenburg": ["Charlotte", "Matthews"],
    "Wake": ["Raleigh", "Cary"],
    "Guilford": ["Greensboro", "High Point"],
    "Wayne": ["Detroit", "Livonia"],
    "Oakland": ["Troy", "Pontiac"],
    "Macomb": ["Warren", "Sterling Heights"]
}

num_records = 1000000
np.random.seed(42)

tax_rates = {}
for county, city_list in cities.items():
    for city in city_list:
        # Assign a random tax rate between 5% and 15% for each city
        tax_rates[city] = np.round(np.random.uniform(0.05, 0.15), 4)

data = []
# Precompute which rows will be 'reclass'
reclass_indices = set(np.random.choice(num_records, 100, replace=False))
for i in range(num_records):
    state_idx = np.random.randint(0, len(states))
    state, counties = states[state_idx]
    county = np.random.choice(counties)
    city = np.random.choice(cities[county])
    tax_rate = tax_rates[city]
    district = f"District-{np.random.randint(1, 100)}"
    gross = np.round(np.random.uniform(100, 10000), 2)
    taxable = np.round(gross * tax_rate, 2)
    if i in reclass_indices:
        place_determination = "reclass"
        input_source = "reclassin"
    else:
        place_determination = "taxareacode"
        input_source = np.random.choice(["balanceimport"])
    entry_id = 100000 + i
    id_val = f"J{100000 + i}"
    log_id = f"L{100000 + i}"
    month = np.random.randint(1, 13)
    year = np.random.choice([2022, 2023, 2024])
    data.append([
        state, city, county, district, gross, taxable,
        input_source, place_determination, entry_id, id_val, log_id, tax_rate, year, month
    ])

columns = [
    "Region", "City", "County", "District", "Gross", "Taxable",
    "InputSource", "PlaceDetermination", "EntryId", "Id", "LogId", "TaxRate", "Year", "Month"
]

df = pd.DataFrame(data, columns=columns)

# For every 'reclassin' record, create a 'reclassout' record with same EntryId and negative Taxable, but different city in the same state
reclassin_mask = df["InputSource"] == "reclassin"
reclassin_rows = df[reclassin_mask].copy()
reclassout_rows = []
balanceimport_rows = []
for idx, row in reclassin_rows.iterrows():
    state = row["Region"]
    county = row["County"]
    original_city = row["City"]
    # Find all possible cities in the state (across all counties)
    counties_in_state = [c for s, cs in states if s == state for c in cs]
    possible_cities = []
    for c in counties_in_state:
        possible_cities.extend(cities[c])
    # Remove the original city from possible choices
    possible_cities = [c for c in possible_cities if c != original_city]
    # If no other city, keep the same city (fallback)
    new_city = np.random.choice(possible_cities) if possible_cities else original_city
    new_tax_rate = tax_rates[new_city]
    # Create reclassout row
    new_row = row.copy()
    new_row["Year"] = row["Year"]
    new_row["Month"] = row["Month"]
    reclassout_row = new_row.copy()
    reclassout_row["InputSource"] = "reclassout"
    reclassout_row["Gross"] = -row["Gross"]
    reclassout_row["Taxable"] = -np.round(abs(row["Gross"]) * new_tax_rate, 2)
    reclassout_row["City"] = new_city
    reclassout_row["TaxRate"] = new_tax_rate
    reclassout_row["Id"] = f"J{100000 + i}"
    reclassout_row["LogId"] = f"J{100000 + i}"
    reclassout_row["Year"] = row["Year"]
    reclassout_row["Month"] = row["Month"]
    reclassout_rows.append(reclassout_row)
    # Create balanceimport row (same as reclassout but InputSource and Taxable positive)
    balanceimport_row = reclassout_row.copy()
    balanceimport_row["InputSource"] = "balanceimport"
    balanceimport_row["Gross"] = abs(balanceimport_row["Gross"])
    balanceimport_row["Taxable"] = abs(balanceimport_row["Taxable"])
    balanceimport_row["Year"] = row["Year"]
    balanceimport_row["Month"] = row["Month"]
    balanceimport_rows.append(balanceimport_row)

if reclassout_rows:
    df = pd.concat([df, pd.DataFrame(reclassout_rows), pd.DataFrame(balanceimport_rows)], ignore_index=True)

df.to_csv("../data/journal_synthetic.csv", index=False)
print("Generated 1,000,000 journal records in ../data/journal_synthetic.csv (with reclassout and balanceimport rows added, including TaxRate)")

Generated 1,000,000 journal records in ../data/journal_synthetic.csv (with reclassout and balanceimport rows added, including TaxRate)
